In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import tool
from langchain import hub
from dotenv import load_dotenv


In [3]:

#I want to import the OPENAI_API_KEY from the .env file
load_dotenv(".env")
if not os.getenv("OPENAI_API_KEY"):
    print("NOT FOUND THE API KEY!")
else:
    print("KEY FOUND!")

KEY FOUND!


In [4]:
try:
    from tools_ocr import extract_text_hybrid_fixed, extract_text_from_image, process_raw_text
    from tools_similarity import calculate_similarity
    from tools_skills import compare_skills_tool
    from tools_courses import get_course_recommendations
except ImportError as e:
    print(f"IMPORT ERROR: {e}")
    print("MAKE SURE THAT THEY EXIST IN THE data/ DIRECTORY")
    exit()


In [ ]:
@tool
def tool_read_image(image_path: str) -> str:
    """
    Sử dụng tool này khi đầu vào là đường dẫn tới file ẢNH (jpg, png, jpeg) của JD hoặc CV.
    Nó sẽ chuyển đổi ảnh thành văn bản.
    Đầu vào: Đường dẫn file ảnh.
    """
    return extract_text_from_image(image_path)

In [5]:
@tool
def tool_process_text_input(raw_text: str) -> str:
    """
    Sử dụng tool này khi người dùng nhập trực tiếp nội dung văn bản (Copy-Paste) thay vì gửi file.
    Nó sẽ làm sạch và chuẩn hóa văn bản.
    Đầu vào: Chuỗi văn bản thô.
    """
    return process_raw_text(raw_text)

In [6]:
@tool
def tool_read_pdf(file_path: str) -> str:
    """
    Sử dụng tool này ĐẦU TIÊN để đọc nội dung văn bản từ file PDF (CV hoặc JD).
    Đầu vào: Đường dẫn tới file PDF (string).
    Đầu ra: Nội dung văn bản thô của file đó.
    """
    # Gọi hàm từ tools_ocr.py
    # Lưu ý: Hàm extract_text_hybrid_fixed cần tham số dpi, lang... ta dùng default
    return extract_text_hybrid_fixed(file_path)

In [7]:
@tool
def tool_calculate_match_score(cv_text: str, jd_text: str) -> float:
    """
    Sử dụng tool này để tính điểm phù hợp (Matching Score) giữa CV và JD.
    Đầu vào: (cv_text, jd_text).
    Đầu ra: Một con số thập phân (ví dụ: 0.75).
    """
    return calculate_similarity(cv_text, jd_text)

In [8]:
@tool
def tool_analyze_skills(cv_text: str, jd_text: str) -> dict:
    """
    Sử dụng tool này để phân tích kỹ năng.
    Nó sẽ trả về danh sách kỹ năng CV có (cv_skills) và kỹ năng CV còn thiếu so với JD (missing_skills).
    Đầu vào: (cv_text, jd_text).
    """
    return compare_skills_tool(cv_text, jd_text)

In [9]:
@tool
def tool_suggest_courses(missing_skills_list: list) -> list:
    """
    Sử dụng tool này SAU KHI đã có danh sách kỹ năng thiếu (missing_skills).
    Nó sẽ gợi ý các khóa học Coursera phù hợp.
    Đầu vào: Danh sách các kỹ năng thiếu (list of strings).
    Đầu ra: Danh sách các khóa học gợi ý.
    """
    # Hàm bên tools_courses cần input là list, đảm bảo agent truyền đúng
    return get_course_recommendations(missing_skills_list)

In [ ]:
# Mình có thể giả sử là JD không phải là PDF mà là kiểu dạng ảnh vì JD thường là ảnh hoặc text
# Nên mình có thể phải tạo thêm 1 tool để preprocess text bởi vì họ có thể đưa text thẳng vào chứ không phải file PDF
# Vậy làm cách nào?
# Mình sẽ tạo thêm 1 tool nữa tên là tool_read_text để đọc text thẳng từ input nếu họ không đưa file PDF


In [10]:
def initialize_agent():
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    tools = [
        tool_read_pdf,
        tool_calculate_match_score,
        tool_analyze_skills,
        tool_suggest_courses,
        tool_read_image,
        tool_process_text_input
    ]
    prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
    return agent_executor


In [11]:
def analyze_cv_jd(cv_path:str, jd_path:str):
    """
    cv_input: Đường dẫn file HOẶC nội dung text
    jd_input: Đường dẫn file HOẶC nội dung text
    cv_type: 'file' (pdf/image) hoặc 'text' (raw text)
    jd_type: 'file' (pdf/image) hoặc 'text' (raw text)
    """
    
    print("STARTING ANALYSIS...")
    agent = initialize_agent()
    user_query = f"""
    Tôi có thông tin CV và JD như sau:
    
    1. CV (Loại: {cv_type}): {cv_input}
    2. JD (Loại: {jd_type}): {jd_input}

    Hãy thực hiện quy trình phân tích chuyên sâu:
    
    BƯỚC 1: Xử lý đầu vào để lấy nội dung văn bản chuẩn.
    - Nếu loại là 'file' và đuôi là .pdf -> Dùng tool_read_pdf.
    - Nếu loại là 'file' và đuôi là .png/.jpg -> Dùng tool_read_image.
    - Nếu loại là 'text' (văn bản thô) -> Dùng tool_process_text_input để làm sạch.
    
    BƯỚC 2: Tính điểm phù hợp (tool_calculate_match_score).
    BƯỚC 3: Phân tích kỹ năng (tool_analyze_skills).
    BƯỚC 4: Gợi ý khóa học (tool_suggest_courses).
    BƯỚC 5: Viết báo cáo tổng hợp.
    """ 
    
    result = agent.invoke({"input": user_query})
    return result['output']

# Mình có thể giả sử là JD không phải là PDF mà là kiểu dạng ảnh vì JD thường là ảnh hoặc text
# Nên mình có thể phải tạo thêm 1 tool để preprocess text bởi vì họ có thể đưa text thẳng vào chứ không phải file PDF
# Vậy làm cách nào?
# Mình sẽ tạo thêm 1 tool nữa tên là tool_read_text để đọc text thẳng từ input nếu họ không đưa file PDF
